In [1]:
import urllib.request
import os, os.path, ssl, urllib, json
import pandas as pd
from tkinter import *
from pathlib import Path
from tkinter import filedialog
from openpyxl import load_workbook
import time
import networkx as nx
from pyvis.network import Network
import datetime
from scipy.optimize import minimize_scalar   
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import random

In [2]:
def objective_function_s2(s2):    
    n=len(rps)    
    gU0 = ((n-1)/n) * sq ** s2
    gUi = sum([gUi**s2 for gUi in rps]) / n
    return abs(gU0-gUi)

def set_s2(rps, sq):
    s2scipy = minimize_scalar(objective_function_s2,[0.1, 0.0000001])
    return s2scipy['x']


def objective_function_s1(s1):
    generalized_transform = sum([(sq**s2 - gUi**s2)/(xi**s1) for gUi,xi in zip(rps, dss)])
    return abs(generalized_transform - 1)

def set_s1(rps, dss, sq, s2):
    s1scipy = minimize_scalar(objective_function_s1, [1, 0.1] )
    return s1scipy['x']

def swing_weights(rps, dss, sq, s2, s1):
    g0 = sq ** s2
    gu = [i**s2 for i in rps]
    gv = [i**s1 for i in dss]
    return [(g0 - i) / j for i,j in zip(gu,gv)]
    #return [ round((g0 - i) / j, 4) for i,j in zip(gu,gv)] # swing weights are traditionally rounded to 4 digits
    

def checks(rps, dss, sq, s2, s1, sw):
    rps = [round(i,2) for i in rps]
    dss = [round(i,2) for i in dss]
    sq = round(sq, 2)
    
    gv = [i**s1 for i in dss]
    xSW = [i*j for i,j in zip(sw,gv)]
    ch_sq = round(sum(xSW) ** (1 / s2), 2)
    ch_rps = []
    for i in range(len(xSW)):
        xSW = [i*j for i,j in zip(sw,gv)]
        xSW[i] = 0
        ch_rps.append( round(sum(xSW) ** (1 / s2), 2) )
    
    status_quo_check = (sq==ch_sq)
    rps_check = all([i==j for i,j in zip(ch_rps, rps)])
    
    return status_quo_check, rps_check#, sq, ch_sq, rps, ch_rps


mytimestamp =  [datetime.datetime.now().year, datetime.datetime.now().month, 
                datetime.datetime.now().day, datetime.datetime.now().hour,
                datetime.datetime.now().minute, datetime.datetime.now().second]
mytimestamp = ''.join([str(i) for i in mytimestamp])


start_time = time.time()

def unique(ll):
    temp = []
    [temp.append(i) for i in ll if i not in temp]
    return temp

def isNaN(num):
    return num != num

def notNaN(num):
    return num == num


In [3]:
print('\nPopulating survey data into the model.')

modeldict = {}

"""
1) start by instantiating all options in the modeldict as mynode options
    - does this model actually need the options as instances in the dictionary? Maybe not.
2) Instantiate all surveys and their X rows into objects
3) Instantiate all groups 
"""
df = pd.read_csv(r'menuCSV.csv') #toymodel.csv')

df.shape

df = df.drop_duplicates(['meal','ingredient', 'group_members']).reset_index()
df.shape

temp = df[df.leaf==1]
options = temp.group_members.dropna().drop_duplicates().tolist()
temp = temp[temp.group_members.isna()].ingredient.drop_duplicates().tolist()
[options.append(i) for i in temp if i not in options]


# - does this model actually need the options as instances in the dictionary? Maybe not.
for option in options:
    dstype = 'leaf'
    modeldict[option] = {'pid':option, 
                      'nodetype':'leaf', 
                      'children':[], 
                      'rps':[], 
                      'sq':'', 
                      'dss':[], 
                      's1':'', 
                      's2':'', 
                      'sw':[],
                      'm1':{},
                      'return_score':''
                      }

temp = df[~df.group_members.isna()]
groups = temp.ingredient.dropna().drop_duplicates().tolist()

df.loc[df.ingredient.isin(groups),'ingredient'] = df.meal + '...' + df.ingredient

temp = df[~df.group_members.isna()]
groups = temp.ingredient.dropna().drop_duplicates().tolist()


df['satisfaction'] =  df['satisfaction'].astype(float) / 100
df['relevance'] =  df['relevance'].astype(float) / 100
df['overall'] =  df['overall'].astype(float) / 100


df_groupby_sw = df[df['relevance_score_type'] == 'impact'].groupby('meal')
pid_children = df_groupby_sw['ingredient'].apply(list)
swing_weight_scores = df_groupby_sw['relevance'].apply(list)
performance_scores  = df_groupby_sw['satisfaction'].apply(list)
status_quo_scores = df_groupby_sw['overall'].apply(list)

"""
Eliminate low impact data sources for products that have >20 data sources

"""

df_groupby_sw = df[df['relevance_score_type'] == 'proportion'].groupby('ingredient')
group_weighted_average_scores = df_groupby_sw['relevance'].apply(list)
group_performance_scores  = df_groupby_sw['satisfaction'].apply(list)
group_children = df_groupby_sw['group_members'].apply(list)

for group in groups:
    groupname = group.split('...')[1]
    #grouptype = src[src['data_source_name'] == groupname]['data_source_type']
    
    modeldict[group] = { 
                      'pid':group, 
                      'nodetype':'group', 
                      'children':group_children[group], 
                      # NOTE: need to normalize group children rps weighted average to decimal out of 1
                      'rps':[i/sum(group_weighted_average_scores[group]) for i in group_weighted_average_scores[group]], 
                      'sq':'', 
                      'dss':group_performance_scores[group], 
                      's1':'', 
                      's2':'',
                      'sw':[],
                      'm1':{},
                      'return_score':''
                      }
    
    
meals = df.meal.dropna().drop_duplicates().tolist()

for meal in  meals: #
    """
    iterate through list of products
    for each product, instantiate node object
    node objet has:
        sq status quo score
        dss data satisfaction score
        rps reduced product score
    """
    
    rps = swing_weight_scores[meal]
    dss = performance_scores[meal]
    children = pid_children[meal]
    sq = status_quo_scores[meal][0]
    
    modeldict[meal] = {'pid':meal, 
                        'nodetype':'meal', 
                        'children':children, 
                        'rps':rps, 
                        'sq':sq, 
                        'dss':dss, 
                      's1':'', 
                      's2':'',
                      'sw':[],
                      'm1':{},
                      'return_score':''
                    }


Populating survey data into the model.


In [4]:
print('\nCalculate swing weight  and spline coefficients.')

"""
Populate the model with s1, s2, swingweight and spline parameters
"""
for key in list(modeldict.keys()):
    
    if modeldict[key]['nodetype'].startswith('meal'):
        """
        Set s2 and s1 values, compute swing weights as sw
        """
        rps = modeldict[key]['rps']
        sq = modeldict[key]['sq']
        dss = modeldict[key]['dss']
        children = modeldict[key]['children']
        #Exception: If there are >20 data sources, the low impact data sources
        # must be removed until there are 20 data sources
        if len(rps) > 20:
            while len(rps) > 20:
                dss = [d for r,d in zip(rps, dss) if r != max(rps)]
                children = [c for r,c in zip(rps, children) if r != max(rps)]
                rps = [r for r in rps if r != max(rps)]
            
            modeldict[key]['rps'] = rps
            modeldict[key]['dss'] = dss
            modeldict[key]['children'] = children
            
            
        
        # Exception of adusting drops that sum to > 150% of sq
        check = 1
        drops = [sq-i for i in rps]
        if sum(drops) > (1.5 * sq):
            while sum(drops) > (1.5 * sq):
                diff = (1.5 * sq) / sum(drops)
                drops = [diff * i for i in drops]
            rps = [sq-i for i in drops]
            check = 0
            
        
        """
        Have seperate conditions for multi data source
        and single data source surveys
        The s1 s2 solver chokes on single data sources, 
        In that case, the default values for s1, s2, sw are all 1
        """                
        
        if len(dss) > 1:
                
            s2=set_s2(rps, sq)
            s1 = set_s1(rps, dss, sq, s2)            
            sw = swing_weights(rps, dss, sq, s2, s1)
            
            if check:
                if checks(rps, dss, sq, s2, s1, sw) != (True, True): 
                    #break
                    print("Check Failed for {}!".format(key))
            
    
            
        elif len(dss) == 1:
            """
            Default to 1 for this special case of single data source surveys
            """
            s1=1
            s2=1
            sw=[1]
            
            """
            Single data source products need to spline the DSS performance score of the source to the SQ score of the parent.
            Such a product should have two m1 spline coefficients: 
                1) the weighted average spline of the group onto itself
                2) the spline of the single X DSS onto the parent SQ score, because IPSEA won't work in this case
            """
            x = dss[0]
            y = sq
            modeldict[key]['m1']['self'] = (y-y*x) / (x-y*x)
            
        else: print("something wrong with " + key)
            
        modeldict[key]['s2'] = s2
        modeldict[key]['s1'] = s1
        modeldict[key]['sw'] = sw
        """
        set spline subdictionary, only need m1 because m2 is 1/m1
        """
        for child, dss in zip(modeldict[key]['children'], modeldict[key]['dss']):
            if modeldict[child]['nodetype'].startswith('meal'):
                x = modeldict[child]['sq']
                y = dss
                modeldict[key]['m1'][child] = (y-y*x) / (x-y*x)
            elif modeldict[child]['nodetype'] in ['group']:
                x = sum([w*s for w,s in zip(modeldict[child]['rps'], modeldict[child]['dss'])]) / sum(modeldict[child]['rps'])
                y = dss
                modeldict[key]['m1'][child] = (y-y*x) / (x-y*x)
                
    """
    set spline subdictionary for products in groups
    """    
    if modeldict[key]['nodetype'] in ['group']:
        for child, dss in zip(modeldict[key]['children'], modeldict[key]['dss']):
            if modeldict[child]['nodetype'].startswith('meal'):
                x = modeldict[child]['sq']
                y = dss
                modeldict[key]['m1'][child] = (y-y*x) / (x-y*x)



Calculate swing weight  and spline coefficients.


In [5]:
def model_analysis(key, rsdict, modeldict, analysis_type='no-drop', programs_series='', network_efficient_product_list='', adjust_item = ''):
    
    """
    a one-size-fits-all function for: 
        the no-drop scores, 
        the drop (or bump) scores for options (plus products, if desired),
        and option combos
        
    """
    
    if modeldict[key]['nodetype'] in ['meal']:
        dss = [i for i in modeldict[key]['dss']]
        s2=modeldict[key]['s2']
        s1 = modeldict[key]['s1']
        sw = [i for i in modeldict[key]['sw'] ]# avoid python overwriting the dictionary entry itself using list comprehension
        
        children = modeldict[key]['children']
        
        # set option to zero if it is the adjust_item
        if analysis_type == 'individual':
            cc=0
            for child in children:
   
                if child==adjust_item: 
                    print(child)
                    dss[cc] = swing_to
                cc+=1

        elif analysis_type == 'combinations':
            cc=0
            for child in children:
                """
                THIS USE OF 'find child in program series set of options' is what makes 
                this section distinct for option combinations
                """
                if child in programs_series[adjust_item]: dss[cc] = swing_to
                cc+=1
                
        return_scores = []
        
        for child, d in zip(children, dss):
            if modeldict[child]['nodetype'] in ['leaf']:
                return_scores.append(d)
            elif modeldict[child]['nodetype'] in ['group', 'meal']:
                
                if child==adjust_item: x = swing_to
                else: x = modeldict[child]['return_score']
                
                if x=='':continue
                m1 = modeldict[key]['m1'][child]
                m2 = 1/m1
                """
                if the spline is a straight line then spline of x is x
                """
                if m1 != m2:
                    spline_result = x + ((m1 * x - x) * (1 + m2 * (x-1)-x)) / ((m1 * x - x) + (1 + m2 * (x-1)-x))
    
                else: spline_result = x
                print(dss, spline_result)
                return_scores.append(spline_result)
        
            if len(return_scores) == len(dss): 
                # if it was even possible to compute the return score because of the availability of already calculated return scores from other data sources
            
                if len(dss) > 1:
            
                    rs = sum([i*j**(s1) for i,j in zip(sw,return_scores)])**(1/s2)
                    modeldict[key]['return_score'] = rs
                    rsdict[key] = rs
                    
                elif len(dss) == 1:
                    
                    if modeldict[child]['nodetype'] in ['leaf']:
                        x = dss[0]        
                        
                    elif modeldict[child]['nodetype'] in ['group', 'meal']:
                        x = modeldict[child]['return_score']
                        # this needs to be splined to the key survey at hand
                            
                        m1 = modeldict[key]['m1'][child]
                        m2 = 1/m1
                        
                        """
                        if the spline is a straight line then spline of x is x
                        """
                        
                        if m1 != 1:
                            x = x + ((m1 * x - x) * (1 + m2 * (x-1)-x)) / ((m1 * x - x) + (1 + m2 * (x-1)-x))
        
                        #else: x = x #redundant
                            
                    if x=='':continue
                    
                    # child is tested for whether it is in the list of the option combo options
                    if child in adjust_item: x = swing_to
                    
                    m1 = modeldict[key]['m1']['self']
                    m2 = 1/m1
                    
                    """
                    if the spline is a straight line then spline of x is x
                    """
                    
                    if m1 != 1:
                        spline_result = x + ((m1 * x - x) * (1 + m2 * (x-1)-x)) / ((m1 * x - x) + (1 + m2 * (x-1)-x))
    
                    else: spline_result = x
                    
                    modeldict[key]['return_score'] = spline_result
                    rsdict[key] = spline_result
                
                if analysis_type == 'no-drop': network_efficient_product_list.append(key)

    
    elif modeldict[key]['nodetype'] in ['group']:
        rps = modeldict[key]['rps']
        dss = [i for i in modeldict[key]['dss']] # avoid python overwriting the dictionary entry itself using list comprehension
        children = modeldict[key]['children']
        
        if analysis_type == 'individual':
            cc=0
            for child in children:
                if child==adjust_item: dss[cc] = swing_to
                cc+=1

        elif analysis_type == 'combinations':
            cc=0
            for child in children:
                """
                THIS USE OF 'find child in program series set of options' is what makes 
                this section distinct for option combinations
                """
                if child in programs_series[adjust_item]: dss[cc] = swing_to
                cc+=1
            
        return_scores = []
        
        for child, d in zip(children, dss):
            if modeldict[child]['nodetype'] in ['leaf']:
                return_scores.append(d)
            elif modeldict[child]['nodetype'] in ['group', 'meal']:

                if child==adjust_item: x = swing_to
                else: x = modeldict[child]['return_score']

                if x=='':continue
                m1 = modeldict[key]['m1'][child]
                m2 = 1/m1
                """
                if the spline is a straight line then spline of x is x
                """
                if m1 != m2:
                    spline_result = x + ((m1 * x - x) * (1 + m2 * (x-1)-x)) / ((m1 * x - x) + (1 + m2 * (x-1)-x))
    
                else: spline_result = x
                return_scores.append(spline_result)
            
            if len(return_scores) == len(dss): 
                # if it was even possible to compute the return score because of the availability of already calculated return scores from other data sources
                rs = sum([w*s for w,s in zip(rps, return_scores)]) / sum(rps)
                modeldict[key]['return_score'] = rs
                rsdict[key] = rs
                
                if analysis_type == 'no-drop': network_efficient_product_list.append(key)
    
   
    if analysis_type == 'no-drop': return rsdict, modeldict, network_efficient_product_list
    
    else: return rsdict, modeldict


In [6]:
"""
Compute return scores in 2 parts
part 1) compute no-drop return scores,
    and as a bonus, make a list of the most efficient path from leaves to root of the network
part 2) using that list of the efficient path up the network,
    compute the drop scores for each option, storing the results in a dictionary
"""



'''
PART 1 - No-Drop Scores
'''

print('\nCompute the no-drop scores.')


return_score_dict = {}
network_efficient_product_list = []

adjust_item=''

rsdict = {}
for k in list(modeldict.keys()): 
    if modeldict[k]['nodetype'] in ['group', 'meal']:
        rsdict[k]=''
        
while '' in rsdict.values():
    # what is left unfilled in the rsdict return score dictionary
    myrange = [k for k,v in rsdict.items() if v == '']
    for key in myrange:
        #print(len(myrange))        
        rsdict, modeldict, network_efficient_product_list = model_analysis(key, rsdict, modeldict, analysis_type='no-drop', network_efficient_product_list=network_efficient_product_list)



# Save modeldict as something to freeze the model as you worked on it; counterpart to a TRE file.
print('\nSaving populated model with no-drop return scores.')
pd.DataFrame(modeldict).T.to_csv(r"DictionaryModel_Test1_modeldict_{}.csv".format(mytimestamp))

"""
Record the results in the return_score_dict ledger
Then, change all return scores back to ''
"""

return_score_dict['No_Drop_Score'] = rsdict

for k in list(modeldict.keys()): 
    modeldict[k]['return_score']=''    
 



Compute the no-drop scores.
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999959277301
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.8499999943798517
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999969117471
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.85
[0.75, 0.76, 0.76] 0.76
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999969117471
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.6999999929728803
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7500000040150434
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.75
[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
[0.75, 0.75] 0.749999993725786
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7000000002919743
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.7499999998915626
[0.8, 0.7, 0.8] 0.7000000064641966
[0.65, 0.65, 0.65, 0.65, 0.65] 0.6499999962529199
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.8, 0.8, 0.8, 0.8] 

In [7]:
"""
PART 2 - Iteratively drop options and products
"""       

#compute_children = options 
#compute_children = products
compute_children = options + meals

swing_to = 0 # swing to zero for compute delta impact analysis
#swing_to = 0.9 # bump test

print('\nIteratively swing to {}.'.format(swing_to))


progress = 0

for adjust_item in compute_children: # Why does it crash with the last option 'CSN' or Chemical Speciation Network???

    print(adjust_item, "{} % analyzed".format(round(100*progress/len(compute_children), 1)) )
    """
    Iterate through adjust_items
    """
    rsdict = {}
    for k in network_efficient_product_list: 
        if modeldict[k]['nodetype'] in ['group', 'meal']:
            rsdict[k]=''
            
    while '' in rsdict.values():
        # what is left unfilled in the rsdict return score dictionary
        myrange = [k for k,v in rsdict.items() if v == '']
        for key in myrange:
            #print(key)
            rsdict, modeldict = model_analysis(key, rsdict, modeldict, analysis_type='individual', adjust_item = adjust_item)

    progress+=1
    """
    Record the results in the return_score_dict ledger
    Then, change all return scores back to ''
    """
    return_score_dict[adjust_item] = rsdict
    
    for k in list(modeldict.keys()): 
        modeldict[k]['return_score']=''
    
    if adjust_item == (compute_children)[-1]: print("Option/Product drop score calculations complete.")



Iteratively swing to 0.
mushrooms 0.0 % analyzed
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
[0.75, 0.75] 0.749999993725786
[0.65, 0.65, 0.65, 0.65, 0.65] 0.6499999962529199
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999959277301
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999910647695
[0.8, 0.8, 0.8, 0.8, 0.8] 0.8000000017130853
[0.8, 0.8, 0.8, 0.8, 0.8] 0.8000000056699661
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999952278087
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.8499999943798517
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999969117471
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999964131064
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7999999910647695
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000074418303
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0

beef 11.1 % analyzed
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
[0.75, 0.75] 0.749999993725786
[0.65, 0.65, 0.65, 0.65, 0.65] 0.6499999962529199
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
beef
[0, 0.8, 0.8, 0.8, 0.8] 0.7999999959277301
[0, 0.8, 0.8, 0.8, 0.8] 0.7999999910647695
[0, 0.8, 0.8, 0.8, 0.8] 0.8000000017130853
[0, 0.8, 0.8, 0.8, 0.8] 0.8000000056699661
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999952278087
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.8499999943798517
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999969117471
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999964131064
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7999999910647695
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000074418303
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 

[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7500000040150434
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.75
[0.7, 0.8, 0.65, 0.7] 0.7999999984259918
[0.75] 0.7499999952278087
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7000000002919743
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7999999943891373
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.7499999998915626
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.7999999943891373
[0.8, 0.7, 0.8] 0.7000000064641966
[0.8, 0.7, 0.8] 0.7999999946544629
[0.7, 0.8, 0.65, 0.7, 0.8, 0.65, 0.7, 0.8, 0.65] 0.7999999959277301
garlic 16.2 % analyzed
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
[0.75, 0.75] 0.749999993725786
[0.65, 0.65, 0.65, 0.65, 0.65] 0.6499999962529199
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999959277301
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999910647695
[0.8, 0.8, 0.8, 0.8, 0.8] 0.8000000017130853
[0

[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.85
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.849999992879738
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8000000018425528
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8299999964668953
[0.75, 0.76, 0.76] 0.749999995326312
[0.75, 0.76, 0.76] 0.7600000009256903
[0.75, 0.76, 0.76] 0.76
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999952278087
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999969117471
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.6999999929728803
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7500000040150434
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.6794582392776521
[0.7, 0.8, 0.65, 0.7] 0.7999999984259918
[0.75] 0.7499999952278087
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7000000002919743
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7999999943891373
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.7499999998915626
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.799999994389137

[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999969117471
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999964131064
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7999999910647695
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000074418303
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.85
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.849999992879738
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8000000018425528
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8299999964668953
[0.75, 0.76, 0.76] 0.749999995326312
[0.75, 0.76, 0.76] 0.7600000009256903
[0.75, 0.76, 0.76] 0.76
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999952278087
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999969117471
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.6999999929728803
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7500000040150434
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.75
[0.7, 0.8, 0.65, 0.7]

...

[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
mustard seed
[0.75, 0.75] 0.749999993725786
[0.65, 0.65, 0.65, 0.65, 0.65] 0.6499999962529199
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999959277301
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999910647695
[0.8, 0.8, 0.8, 0.8, 0.8] 0.8000000017130853
[0.8, 0.8, 0.8, 0.8, 0.8] 0.5734693931563357
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999952278087
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.8499999943798517
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999969117471
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999964131064
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7999999910647695
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0.8, 0.7, 0.7] 0.468290264423167
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.85
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.849999992

[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999964131064
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7999999910647695
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000074418303
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.85
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.849999992879738
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8000000018425528
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8299999964668953
[0.75, 0.76, 0.76] 0.749999995326312
[0.75, 0.76, 0.76] 0.7600000009256903
[0.75, 0.76, 0.76] 0.76
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999952278087
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999969117471
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.6999999929728803
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7500000040150434
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.75
[0.7, 0.8, 0.65, 0.7] 0.7999999984259918
[0.75] 0.7499999952278087
[0.7, 0.9

[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
[0.75, 0.75] 0.749999993725786
mayonaise
[0.65, 0, 0.65, 0.65, 0.65] 0.0
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999959277301
[0.8, 0.8, 0.8, 0.8, 0.8] 0.7999999910647695
[0.8, 0.8, 0.8, 0.8, 0.8] 0.8000000017130853
[0.8, 0.8, 0.8, 0.8, 0.8] 0.8000000056699661
mayonaise
[0.75, 0.85, 0.8, 0.75, 0, 0.75] 0.7499999952278087
[0.75, 0.85, 0.8, 0.75, 0, 0.75] 0.8499999943798517
[0.75, 0.85, 0.8, 0.75, 0, 0.75] 0.0
[0.75, 0.85, 0.8, 0.75, 0, 0.75] 0.7499999964131064
[0.7, 0.65, 0.8, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7999999910647695
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0.8, 0.7, 0.7] 0.7000000074418303
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.7999999937529688
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.85
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85

[0.7, 0.8, 0.65, 0.7, 0.8, 0.65, 0.7, 0.8, 0.65] 0.7999999959277301
french fries 87.9 % analyzed
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
[0.8, 0.65, 0.7, 0.8] 0.6999999929728803
[0.65, 0.7, 0.8, 0.65, 0.7, 0.8] 0.799999994646004
[0.75, 0.75] 0.749999993725786
[0.65, 0.65, 0.65, 0.65, 0.65] 0.6499999962529199
[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65] 0.649999992387287
french fries
[0.8, 0.8, 0, 0.8, 0.8] 0.7999999959277301
[0.8, 0.8, 0, 0.8, 0.8] 0.0
[0.8, 0.8, 0, 0.8, 0.8] 0.8000000017130853
[0.8, 0.8, 0, 0.8, 0.8] 0.8000000056699661
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999952278087
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.8499999943798517
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.7499999969117471
[0.75, 0.85, 0.8, 0.75, 0.75, 0.75] 0.2006239828834336
french fries
[0.7, 0.65, 0, 0.7, 0.7] 0.6499999942097412
[0.7, 0.65, 0, 0.7, 0.7] 0.0
[0.7, 0.65, 0, 0.7, 0.7] 0.7000000013759178
[0.7, 0.65, 0, 0.7, 0.7] 0.7000000074418303
french fries
[0.8, 0.8, 0.8, 0.85, 0.75, 0, 

[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.849999992879738
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8000000018425528
[0.8, 0.8, 0.8, 0.85, 0.75, 0.85, 0.8, 0.83] 0.8299999964668953
[0.75, 0.76, 0.76] 0.749999995326312
[0.75, 0.76, 0.76] 0.7600000009256903
[0.75, 0.76, 0.76] 0.76
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999952278087
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7499999969117471
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.6999999929728803
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.7500000040150434
[0.75, 0.7, 0.75, 0.7, 0.75, 0.7, 0.75] 0.75
ice cream
[0.7, 0, 0.65, 0.7] 0
[0.75] 0.7499999952278087
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7000000002919743
[0.7, 0.95, 0.8, 0.7, 0.8] 0.7999999943891373
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.7499999998915626
[0.75, 0.6, 0.8, 0.8, 0.8, 0.8, 0.8, 0.7, 0.75, 0.8] 0.7999999943891373
[0.8, 0.7, 0.8] 0.7000000064641966
[0.8, 0.7, 0.8] 0.7999999946544629
[0.7, 0.8, 0.65, 0.7, 0.8, 0.65, 0.7, 0.8, 0.65] 0.79999999592773

In [8]:
kps = meals

[return_score_dict[opt][k] for k in kps for opt in options]


# 4) Shape this into something that resembles ComputeDelta raw output as a dataframe, save to CSV
df = pd.DataFrame(return_score_dict).T
df_kps = df[kps]
df_kps_raw_impact = (df_kps.iloc[0,:] - df_kps) / df_kps.iloc[0,:]  # subtract everything from the satisfaction scores to get raw impact
df_kps_raw_impact.iloc[0,:] = df_kps.iloc[0,:] # replace the raw no-drop scores back into the first row

df_kps_raw_impact

df_kps_raw_impact = df_kps_raw_impact.iloc[1:,:]# get rid of the original satisfaction socre score
df_kps_raw_impact.to_csv(r'impact_results_{}.csv'.format(mytimestamp)) 


In [9]:
# Make a DiGraph out of edges. The edges are tuples of nodes
edges = []
weights = []
for parent in df_kps_raw_impact.columns:
    for child in df_kps_raw_impact.index:
        weight = df_kps_raw_impact[parent].loc[child]
        if weight  != 0:
            edges.append( tuple([parent, child]) )
            weights.append(weight )
            

G = nx.DiGraph(edges)

# output a JPEG image of the network graph

f = plt.figure(figsize=(10,10))
# found a layout that works for you. shell works well, and kamada_kawai may also work
pos = nx.shell_layout(G) # pos = nx.kamada_kawai_layout(G)

nx.draw(G, pos, width = weights, edge_color='navy', node_color = 'blue', with_labels = False)
text = nx.draw_networkx_labels(G,pos)
for _,t in text.items():
    t.set_rotation(random.randint(-10,10)) # jitter label angle of rotation

f.savefig(r"portfolio_network_image_{}.png".format(mytimestamp))

In [10]:
# make a react graph as an HTML
nt = Network(height='750px', width='100%', bgcolor='#222222', font_color='white', directed =True)
# loop through names of nodes to add nodes
c = 0
for parent in df_kps_raw_impact.columns:
    for child in df_kps_raw_impact.index:
        weight = df_kps_raw_impact[parent].loc[child]
        if weight  != 0:
            
            if parent not in [i['title'] for i in nt.nodes]:
                nt.add_node(c, parent, title=str(parent), color='cyan')
                c+=1
            if child not in [i['title'] for i in nt.nodes]:
                nt.add_node(c, child, title=str(child), color='cyan')
                c+=1

            parent_id = [i for i in nt.nodes if i['title']==parent][0]['id']
            child_id = [i for i in nt.nodes if i['title']==child][0]['id']

            nt.add_edge(child_id, parent_id, color='orange', width = weight*25 )

# populates the nodes and edges data structures
nt.show(r"portfolio_network_image_{}.html".format(mytimestamp))


In [11]:
from IPython.display import IFrame

IFrame(src='./portfolio_network_image_example.html', width=700, height=600)